# Forecasting and Analysis using `fbprophet`

This notebook integrates time series analysis and forecasting using Google search trends and stock price data.

In [ ]:

# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
from fbprophet import Prophet

# Load Google search trends data (mock paths provided for demonstration)
search_data = pd.read_csv('google_search_data.csv', parse_dates=True, index_col='date')

# Load stock price data
stock_data = pd.read_csv('stock_price_data.csv', parse_dates=True, index_col='date')

# Display a preview of the data
print("Google Search Trends Data:")
print(search_data.head())

print("\nStock Price Data:")
print(stock_data.head())


In [ ]:

# Analyze Google search traffic patterns
# Group by hour of the day
avg_hourly_traffic = search_data.groupby(search_data.index.hour).mean()['search_traffic']
avg_hourly_traffic.plot(title="Average Hourly Google Search Traffic", figsize=(10,6))
plt.show()

# Group by day of the week
avg_daily_traffic = search_data.groupby(search_data.index.dayofweek).mean()['search_traffic']
avg_daily_traffic.plot(title="Average Daily Google Search Traffic", figsize=(10,6))
plt.show()

# Group by week of the year
avg_weekly_traffic = search_data.groupby(search_data.index.week).mean()['search_traffic']
avg_weekly_traffic.plot(title="Average Weekly Google Search Traffic", figsize=(10,6))
plt.show()


In [ ]:

# Combine search traffic and stock price data
combined_data = pd.concat([search_data['search_traffic'], stock_data['stock_price']], axis=1)

# Plot combined data for the first half of 2020
first_half_2020 = combined_data.loc['2020-01':'2020-06']

fig, ax1 = plt.subplots(figsize=(10, 6))
ax1.plot(first_half_2020.index, first_half_2020['search_traffic'], color='tab:blue', label='Search Traffic')
ax2 = ax1.twinx()
ax2.plot(first_half_2020.index, first_half_2020['stock_price'], color='tab:red', label='Stock Price')

ax1.set_xlabel('Date')
ax1.set_ylabel('Search Traffic', color='tab:blue')
ax2.set_ylabel('Stock Price', color='tab:red')
plt.title('Google Search Traffic and Stock Price (Jan to Jun 2020)')
plt.show()

# Calculate lagged search traffic and stock volatility
combined_data['Lagged_Search_Traffic'] = combined_data['search_traffic'].shift(1)
combined_data['Stock_Volatility'] = combined_data['stock_price'].ewm(span=4).std()
combined_data['Hourly_Stock_Return'] = combined_data['stock_price'].pct_change()

# Display processed data
print(combined_data.head())


In [ ]:

# Prepare data for Prophet
prophet_data = search_data.reset_index()[['date', 'search_traffic']].rename(columns={'date': 'ds', 'search_traffic': 'y'})

# Initialize and fit the Prophet model
model = Prophet()
model.fit(prophet_data)

# Create a future dataframe
future = model.make_future_dataframe(periods=365, freq='D')

# Predict future search traffic
forecast = model.predict(future)

# Plot the forecast
model.plot(forecast)
plt.title('Prophet Forecast of Google Search Traffic')
plt.show()

# Plot the individual components of the forecast
model.plot_components(forecast)
plt.show()


### Questions and Answers

**Q1:** What time of day exhibits the greatest popularity?
**A1:** Answer based on the hourly analysis plot.

**Q2:** Which day of the week gets the most search traffic?
**A2:** Answer based on the daily analysis plot.

**Q3:** What's the lowest point for search traffic in the calendar year?
**A3:** Answer based on the Prophet forecast and seasonality analysis.
